<a href="https://colab.research.google.com/github/chiranjeevi-manike/IBM-Cognitiveclass.ai/blob/main/cancer_type.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!git clone https://github.com/zigzagktz/Cancer-Classification-by-Gene-Expression-using-SVM-and-NaiveBayes

Cloning into 'Cancer-Classification-by-Gene-Expression-using-SVM-and-NaiveBayes'...
remote: Enumerating objects: 96, done.
remote: Total 96 (delta 0), reused 0 (delta 0), pack-reused 96
Unpacking objects: 100% (96/96), done.


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
import time
from sklearn.metrics import accuracy_score,  f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score
import random
import time
%matplotlib inline

In [9]:
random.seed(1438642)
df = pd.read_csv("/content/Cancer-Classification-by-Gene-Expression-using-SVM-and-NaiveBayes/train.csv")
patient_number = pd.read_csv("/content/Cancer-Classification-by-Gene-Expression-using-SVM-and-NaiveBayes/actual.csv")
patient_number.columns = ["index","cancer_type"]
test = pd.read_csv("/content/Cancer-Classification-by-Gene-Expression-using-SVM-and-NaiveBayes/test.csv")

In [10]:
columns_names = [col for col in df.columns if "call" not in col]
df1 = df[columns_names]
df2 = df1.drop(["Gene Description","Gene Accession Number"],axis=1)
df2  = df2.T

df2 = df2.reset_index()
df2["index"] = pd.to_numeric(df2["index"])
df2 = df2.sort_values("index")

df2  = pd.merge(df2, patient_number, on='index', how='inner')
df2.loc[df2.cancer_type=="ALL","cancer_type"] = 0
df2.loc[df2.cancer_type=="AML","cancer_type"] = 1
df2 = df2.sample(frac=1).drop(["index"],axis=1)

In [11]:
df2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,7090,7091,7092,7093,7094,7095,7096,7097,7098,7099,7100,7101,7102,7103,7104,7105,7106,7107,7108,7109,7110,7111,7112,7113,7114,7115,7116,7117,7118,7119,7120,7121,7122,7123,7124,7125,7126,7127,7128,cancer_type
6,-72,-144,238,55,-399,-551,131,-179,126,-20,-115,-1003,-761,-541,109,435,-129,-399,30801,43,80,-87,-8,-250,100,-73,-132,-57,-52,-73,-327,-28,-613,-317,-220,-179,58,383,21573,11031,...,-618,379,280,351,211,20531,18708,1828,452,720,9542,7144,835,993,-361,-68,448,-16,317,67,36,546,304,243,349,312,-488,-2300,552,30,819,-178,151,-18,1140,482,10,369,-42,0
11,-67,-93,84,25,-179,-323,-135,-127,-2,-66,208,-472,-163,-192,-80,89,-106,-251,27087,-5,186,-18,-2,-150,79,-57,-116,-45,38,-79,-165,15,-432,-231,-321,-224,-17,152,8299,4204,...,761,208,319,235,1434,14924,23030,685,370,175,11347,10249,352,3158,-271,-47,120,-27,126,-24,2,-287,137,162,181,91,-281,-1399,219,48,224,60,194,-10,291,41,8,-2,-80,0
36,-25,-20,124,325,-396,-464,-221,-390,-1,358,-197,-852,-689,-175,-7,-156,-47,-397,17359,44,286,-73,-11,-272,118,-88,-206,-5,-68,-60,-278,38,-463,-228,10,1287,810,1378,2183,1563,...,962,425,418,479,691,17484,16806,1253,328,454,13591,13307,391,2441,-396,-51,228,-50,197,54,74,222,211,99,251,249,-430,-3452,393,146,486,-32,334,35,1733,304,12,3193,-33,1
18,-44,-51,100,207,-146,-221,83,-198,34,-20,11,-535,-386,-64,49,13,-46,-334,20225,-42,191,-55,-29,-174,-3,-123,-99,-18,-57,-76,-154,20,-736,-391,-423,-344,126,370,-56,-284,...,1274,133,250,293,171,12040,15675,602,310,222,10248,7370,281,3243,-173,73,272,-14,153,10,51,-24,156,89,142,204,-147,-728,1660,318,397,-48,1939,-18,538,90,-50,333,-24,0
10,-165,-155,-71,82,-163,-430,100,-109,56,350,204,-524,-599,-188,34,221,-41,-204,18167,-8,115,-95,14,-54,91,-81,-134,22,5,-14,-269,42,-509,-233,-275,-276,46,266,791,122,...,-473,415,227,704,43,18573,17651,816,175,9,6012,4479,187,817,-232,-25,150,-28,306,114,97,44,177,195,192,173,-244,-1395,1972,185,642,32,137,-81,672,208,25,116,-62,0


In [12]:

columns_names_test = [col for col in test.columns if "call" not in col]
test1 = test[columns_names_test]
test2 = test1.drop(["Gene Description","Gene Accession Number"],axis=1)
test2  = test2.T
test2 = test2.reset_index()
test2["index"] = pd.to_numeric(test2["index"])
test2 = test2.sort_values("index")
test2  = pd.merge(test2, patient_number, on='index', how='inner')

test2.loc[test2.cancer_type=="ALL","cancer_type"] = 0
test2.loc[test2.cancer_type=="AML","cancer_type"] = 1

test2 = test2.sample(frac=1).drop(["index"],axis=1)

In [14]:
#first Support vector machine classifier with 7000 variables
time_laps = []
for i in range(100):
    start_time = time.time()
    clf = svm.SVC(kernel='linear')
    clf.fit(df2.iloc[:,0:df2.shape[1]],df2["cancer_type"])
    pred = clf.predict(test2.iloc[:,0:test2.shape[1]])
    accuracy = accuracy_score(test2.cancer_type,pred)
    laps = time.time() - start_time
    time_laps.append(laps)
avg_time_laps = np.mean(time_laps)
print("The accuracy is {} and computation time is {}".format(accuracy,avg_time_laps))
SVM_accuracy = accuracy
SVM_computation_time = avg_time_laps

ValueError: ignored

In [15]:
#first Naive Bayse model and computation time
com_time = []
for i in range(100):
    start_time = time.time()
    nv = GaussianNB()
    nv.fit(df2.iloc[:,0:df2.shape[1]],df2["cancer_type"])
    pred2 = nv.predict(test2.iloc[:,0:test2.shape[1]])
    accuracy = accuracy_score(test2.cancer_type,pred2)
    time_laps = time.time() -start_time
    com_time.append(time_laps)
    avg_comp_time = np.mean(com_time)
print("The accuracy is {} and computation time is {}".format(accuracy,avg_comp_time))
NB_accuracy = accuracy
NB_computation_time = avg_comp_time

ValueError: ignored

In [ ]:
lis = [SVM_accuracy*100,SVM_computation_time]
lis2= [NB_accuracy*100,NB_computation_time]

In [ ]:
cross_tab = pd.DataFrame(lis,lis2)
cross_tab.reset_index(inplace=True)
cross_tab.columns=("Naive Bayes","SVM")
cross_tab.index=("Accuracy","Computation time")
print("    Metric with 7128 columns")
cross_tab

In [ ]:
import warnings
warnings.filterwarnings("ignore")

start_time = time.time()
# Normalizing the full dataframe
new = pd.concat([df2,test2],axis=0)
standard = StandardScaler()
result = standard.fit_transform(new.iloc[:,0:new.shape[1]])
result = pd.DataFrame(result)

# PCA calculation
pca = PCA(n_components=70)
principal_comp = pca.fit_transform(result)
pca_time = time.time()-start_time


principal_comp = pd.DataFrame(principal_comp)
explanation = pca.explained_variance_ratio_
cummulative_sum = explanation.cumsum()

print("Time to compute PCA {}".format(pca_time))


plt.figure(figsize=(14,6))
plt.plot(cummulative_sum*100,linewidth=4)
plt.xlabel("Number of Variables in Use",size=15,color="darkred")
plt.ylabel("Variability Explained in %",size=15,color="darkred")
plt.title("Almost 99% Variablity explained by 70 Variables",size=20,color="darkred")

In [ ]:
y_col = pd.concat([df2.cancer_type,test2.cancer_type],axis=0)
principal_comp = pd.DataFrame(principal_comp)
principal_comp = pd.concat([principal_comp,y_col.reset_index()],axis=1).drop("index",axis=1)
train_x,test_x,train_y,test_y = train_test_split(principal_comp.iloc[:,0:principal_comp.shape[1]],principal_comp.cancer_type,test_size=0.25)

In [ ]:
# SVM model after PCA
com_time_laps=[]
for i in range(100):
    start_time = time.time()
    clf2 = svm.SVC()
    clf2.fit(train_x,train_y)
    accuracy = accuracy_score(clf2.predict(test_x),test_y) * 100
    accuracy=round(accuracy)
    time_laps_after_pca = time.time() - start_time
    com_time_laps.append(time_laps_after_pca)
print("The accuracy is {} and computaiton time is {} \nNeeds Improvement in accuracy".format(accuracy,np.mean(com_time_laps)))
lis = [accuracy,np.mean(com_time_laps)]

In [ ]:
#Total change in computaion time
x = ((avg_time_laps-time_laps_after_pca )/avg_time_laps ) *100
print("Overall Improvement in computation of SVM is: \x1b[1;31m {} \x1b[0m ".format(round(x)))

In [ ]:
train_x.head()

In [ ]:
start_time = time.time()
nv2 = GaussianNB()
nv2.fit(train_x.iloc[:,0:train_x.shape[1]-1],train_y)
time_laps = time.time()-start_time
a = accuracy_score(nv2.predict(test_x.iloc[:,0:test_x.shape[1]-1]),test_y)*100
naive_bayse = round(a)

lis2=[naive_bayse,time_laps]

In [ ]:
cross_tab = pd.DataFrame(lis,lis2)
cross_tab.reset_index(inplace=True)
cross_tab.columns=("Naive Bayes","SVM")
cross_tab.index=("Accuracy","Computation time")
print("    Metric with 72 columns")
cross_tab

In [ ]:
acc = []
acc_nb = []
f1_svm = []
f1_nb = []
for i in range(50,72):
    pca = PCA(n_components=i)
    principal_comp = pca.fit_transform(result)
    principal_comp = pd.DataFrame(principal_comp)
    y_col = pd.concat([df2.cancer_type,test2.cancer_type],axis=0)
    principal_comp = pd.DataFrame(principal_comp)
    principal_comp = pd.concat([principal_comp,y_col.reset_index()],axis=1).drop("index",axis=1)

    #SVM
    train_x,test_x,train_y,test_y = train_test_split(principal_comp.iloc[:,0:(principal_comp.shape[1]-1)],principal_comp.cancer_type,test_size=0.45)
    
    clf2 = svm.SVC(kernel='linear',gamma="auto",C=0.001)
    clf2.fit(train_x,train_y)
    a = accuracy_score(clf2.predict(test_x),test_y) * 100
    
    f1_ = f1_score(test_y, clf2.predict(test_x))
    f1_svm.append(f1_)
    
    
    #NB
    nv2 = GaussianNB()
    nv2.fit(train_x,train_y)
    b = accuracy_score(nv2.predict(test_x),test_y) *100
    
    f1 = f1_score(test_y, clf2.predict(test_x))
    f1_nb.append(f1)

    acc.append(a)
    acc_nb.append(b)
    
    
plt.figure(figsize=(15,6))
plt.plot(range(50,72),acc,'#ff7f0e',range(50,72),acc_nb,'#1f77b4',linewidth=5)
plt.legend(["Support Vector Machine","Naive Bayes"],loc="best")
plt.xlabel("Number of Features Selected",size=15)
plt.ylabel("Accuracy in Percentage %",size=15)
plt.title("Comparision of SVM and NaiveBayes as Feature Grows",size=25)
plt.style.use("bmh")

In [ ]:
#hyper parameter at var=72
from sklearn.metrics import f1_score

kernel=["linear","poly","rbf","sigmoid"]
cost = [.001,.01,.1,1,10,100,1000]

m = []
f= []

for k in kernel:
    ker = k
    for c in cost:
        cos = c
        accu=[]
        f1_scores = []
        
        for i in range(10):
            train_x,test_x,train_y,test_y = train_test_split(principal_comp.iloc[:,0:(principal_comp.shape[1]-1)],principal_comp.cancer_type,test_size=0.45)
            clf2 = svm.SVC(kernel=ker,gamma="auto",C=cos)
            clf2.fit(train_x,train_y)
            a = accuracy_score(clf2.predict(test_x),test_y)             
            accu.append(a)
            
            from sklearn.metrics import f1_score
            f1 = f1_score(test_y,clf2.predict(test_x),average='weighted')
            f1_scores.append(f1)
            
        means = np.mean(accu)
        f1_means = np.mean(f1_scores)
        f.append(f1_means)
        m.append(means)     
        print(" {} kernel,  penalty = {}, accuacy = {}, f1_score = {} ".format(k,c,round(means*100,2),f1_means))
        time.sleep(.3)